In [ ]:
%pip install streamlit

In [4]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import time
import json
import streamlit as st

In [5]:
# 환경변수 로드
load_dotenv()

True

# 함수 정의

In [6]:
def __init__(self):
    self.client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
    self.assistant_id = os.getenv('ASSISTANT_ID')
    self.vector_store_id = os.getenv('VECTOR_STORE_ID')


In [ ]:
def upload_file_to_assistant(self, file_path):
        """
        파일 업로드 후 벡터 스토어에 추가
        """
        try:
            with open(file_path,'rb') as f:
                uploaded_file = self.client.files.create(
                    file=f,
                    purpose='assistants'
                )

            self.client.beta.vector_stores.files.create(
                vector_stroe_id = self.vector_store_id,
                file_id=uploaded_file.id
            )

            return uploaded_file.id
        
        except Exception as e:
            st.error(f"---ERROR for file uploading---")
            return None

In [ ]:
def create_thread_and_run(self, uploaded_files, prompt):
    """
    스레드 생성, Assistant 실행
    """
    try:
        # 스레드 생성
        thread = self.client.beta.threads.create()

        message_content = prompt

        if uploaded_files:
            message_content += f"\n\n첨부된 파일들을 분석하여 TTL 파일을 생성해주세요."

        message = self.client.beta.threads.message.create(
            thread_id = thread.id,
            rold = "user",
            content=message_content,
            attachments = [{"file_id": file_id, 
                            "tools":[{"type": "file_search"}]}
                            for file_id in uploaded_files] if uploaded_files else []
        )

        run = self.client.beta.threads.runs.create(
            thread_id=thread.id,
            assisant_id=self.assitant_id,
            additional_instructions="목차를 기준으로 다른 문서들을 분석하여 OWL/RDF 표준에 맞는 TTL 파일을 생성해주세요. TTL 파일은 Turtle 형식으로 작성하고, 적절한 네임스페이스와 클래스, 속성을 정의해주세요."
        )

        return thread.id, run.id

    except Exception as e:
        st.error(f"---Error for create and run threads: {str{e}} ")
        return None, None